In [1]:
library(cowplot)
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(scales)
library(repr)
library(stargazer)
library(doMC)
registerDoMC(12)


********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor



Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Loading required package: foreach

Loading required package: iterators

Loading 

In [2]:
panel <- readRDS('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_v2.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/conley_selector_matrix.Rdata')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/social_adj_matrix.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/geo_adjacency_matrix.Rdata')
source('../utils/residualizer+genAlters.R')
source("../utils/labels_and_colors.r")
source('../utils/adjacency_and_cluster_robust_inference.R')
source('../utils/geo_utils.r')

Loaded glmnet 3.0-2



Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘lmtest’


The following object is masked from ‘package:lfe’:

    waldtest




In [3]:
panel %>%
    select(-matches('iv...'),
           -matches('alter'),
           -matches('shiftshare'),
           -PRCP, -TMAX) -> panel

In [4]:
getCoef <- function(df, DV, type = 'noSP', i) {
    if(type == 'noSP') {
        form <-str_c(DV, '.r ~ 0 + stay_home.r + ban_gmr.r | county_fips + ds | 0 | state_abbv')
    } else if (type == 'geoOnly') {
            form <- str_c(DV, '.r ~ 0 + stay_home.r + geo_alter_sh.r + 
                                    ban_gmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv')
    } else {
            form <- str_c(DV, '.r ~ 0 + stay_home.r + alter_sh.r + geo_alter_sh.r + 
                                    ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv')
    }
    
    fit <- felm(as.formula(form), df, weights = df$n)
    fp <- str_c('/nfs/sloanlab004/projects/covid_mobility_proj/models/did_placebo/DiD_', type, '_', DV, '_', str_pad(i, 4, pad = '0'), '.RDS')
    saveRDS(fit, fp)
    data.frame(summary(fit)$coef) %>%
        mutate(vars = rownames(.),
               DV = DV,
               type = type) %>%
        select(DV, type, vars, coef = Estimate) -> out
    return(out)
}


In [6]:
placebo <- function(i) {
    set.seed(i)
    panel %>%
        select(county_fips) %>%
        distinct() %>%
        bind_cols(panel %>% 
                  select(county_fips) %>% 
                  distinct() %>%
                  sample_n(n()) %>%
                  rename(swap = county_fips)) %>%
        left_join(panel %>%
                  select(swap = county_fips, ds, ban_gmr, stay_home)) %>%
        arrange(ds, county_fips) -> policySwap

    panel %>%
        mutate(ban_gmr    = policySwap$ban_gmr,
               stay_home  = policySwap$stay_home) %>%
        mutate(alter_bgmr     = weightedAlters(., WM, ban_gmr),
               alter_sh       = weightedAlters(., WM, stay_home),
               geo_alter_bgmr = weightedAlters(., gWM, ban_gmr),
               geo_alter_sh   = weightedAlters(., gWM, stay_home)) %>%
        mutate(ban_gmr.r        = panel_residualizer(ban_gmr),
               stay_home.r      = panel_residualizer(stay_home),
               alter_bgmr.r     = panel_residualizer(alter_bgmr),
               alter_sh.r       = panel_residualizer(alter_sh),
               geo_alter_bgmr.r = panel_residualizer(geo_alter_bgmr),
               geo_alter_sh.r   = panel_residualizer(geo_alter_sh)) -> df
    
    out <- foreach(DV = c('ash_rnstu', 'ash_nhd', 'fb_btvrc', 'log_mcbgv'), .combine = rbind) %:%
        foreach(type = c('noSP', 'geoOnly', 'geoSocial'), .combine = rbind) %dopar% getCoef(df, DV, type, i)
    return(out)
}
placebo(1)

Joining, by = "swap"



[1]	train-rmse:0.290349+0.001510	test-rmse:0.290313+0.005128 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.193084+0.001512	test-rmse:0.193320+0.005465 
[3]	train-rmse:0.159249+0.001666	test-rmse:0.160468+0.004983 
[4]	train-rmse:0.149265+0.001756	test-rmse:0.151245+0.004402 
[5]	train-rmse:0.146306+0.001750	test-rmse:0.149025+0.004120 
[6]	train-rmse:0.145358+0.001673	test-rmse:0.148695+0.004118 
[7]	train-rmse:0.144888+0.001627	test-rmse:0.148589+0.004104 
[8]	train-rmse:0.144560+0.001581	test-rmse:0.148687+0.004132 
[9]	train-rmse:0.144199+0.001513	test-rmse:0.148816+0.004061 
[10]	train-rmse:0.143844+0.001442	test-rmse:0.148996+0.003913 
Stopping. Best iteration:
[7]	train-rmse:0.144888+0.001627	test-rmse:0.148589+0.004104

[1]	train-rmse:0.303710+0.000514	test-rmse:0.303710+0.001467 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rm

DV,type,vars,coef
<chr>,<chr>,<chr>,<dbl>
ash_rnstu,noSP,stay_home.r,7.696210e-04
ash_rnstu,noSP,ban_gmr.r,-9.719204e-04
ash_rnstu,geoOnly,stay_home.r,7.642494e-04
ash_rnstu,geoOnly,geo_alter_sh.r,1.117324e-02
ash_rnstu,geoOnly,ban_gmr.r,-9.999843e-04
ash_rnstu,geoOnly,geo_alter_bgmr.r,-8.256640e-03
ash_rnstu,geoSocial,stay_home.r,1.259558e-03
ash_rnstu,geoSocial,alter_sh.r,7.469586e-02
ash_rnstu,geoSocial,geo_alter_sh.r,-1.104923e-02


In [ ]:
placeboResults <- foreach(i = 1:1000, .combine = rbind) %do% placebo(i)

Joining, by = "swap"



[1]	train-rmse:0.290349+0.001510	test-rmse:0.290313+0.005128 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.193084+0.001512	test-rmse:0.193320+0.005465 
[3]	train-rmse:0.159249+0.001666	test-rmse:0.160468+0.004983 
[4]	train-rmse:0.149265+0.001756	test-rmse:0.151245+0.004402 
[5]	train-rmse:0.146306+0.001750	test-rmse:0.149025+0.004120 
[6]	train-rmse:0.145358+0.001673	test-rmse:0.148695+0.004118 
[7]	train-rmse:0.144888+0.001627	test-rmse:0.148589+0.004104 
[8]	train-rmse:0.144560+0.001581	test-rmse:0.148687+0.004132 
[9]	train-rmse:0.144198+0.001513	test-rmse:0.148816+0.004061 
[10]	train-rmse:0.143844+0.001442	test-rmse:0.148996+0.003913 
Stopping. Best iteration:
[7]	train-rmse:0.144888+0.001627	test-rmse:0.148589+0.004104

[1]	train-rmse:0.303710+0.000514	test-rmse:0.303710+0.001467 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rm

Joining, by = "swap"



[1]	train-rmse:0.290365+0.002470	test-rmse:0.290499+0.007506 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.193086+0.002707	test-rmse:0.193771+0.008264 
[3]	train-rmse:0.159233+0.003051	test-rmse:0.160806+0.007780 
[4]	train-rmse:0.149274+0.003158	test-rmse:0.151412+0.007153 
[5]	train-rmse:0.146500+0.003231	test-rmse:0.149028+0.006665 
[6]	train-rmse:0.145419+0.003213	test-rmse:0.148319+0.006529 
[7]	train-rmse:0.144739+0.003228	test-rmse:0.148276+0.006239 
[8]	train-rmse:0.144328+0.003196	test-rmse:0.148450+0.006102 
[9]	train-rmse:0.143894+0.003088	test-rmse:0.148653+0.005996 
[10]	train-rmse:0.143578+0.003165	test-rmse:0.148877+0.006085 
Stopping. Best iteration:
[7]	train-rmse:0.144739+0.003228	test-rmse:0.148276+0.006239

[1]	train-rmse:0.303683+0.000679	test-rmse:0.303774+0.005067 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rm

Joining, by = "swap"



...

Joining, by = "swap"



[1]	train-rmse:0.290374+0.001362	test-rmse:0.290452+0.005055 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.193067+0.001283	test-rmse:0.193645+0.005021 
[3]	train-rmse:0.159346+0.001442	test-rmse:0.160371+0.004220 
[4]	train-rmse:0.149394+0.001444	test-rmse:0.151100+0.003591 
[5]	train-rmse:0.146583+0.001569	test-rmse:0.148640+0.003281 
[6]	train-rmse:0.145598+0.001527	test-rmse:0.148410+0.003033 
[7]	train-rmse:0.145116+0.001669	test-rmse:0.148261+0.003004 
[8]	train-rmse:0.144730+0.001652	test-rmse:0.148463+0.003018 
[9]	train-rmse:0.144406+0.001636	test-rmse:0.148671+0.002936 
[10]	train-rmse:0.144026+0.001834	test-rmse:0.148840+0.002972 
Stopping. Best iteration:
[7]	train-rmse:0.145116+0.001669	test-rmse:0.148261+0.003004

[1]	train-rmse:0.303430+0.001191	test-rmse:0.303941+0.004991 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rm

Joining, by = "swap"



[1]	train-rmse:0.290392+0.001012	test-rmse:0.290378+0.003943 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.193158+0.000881	test-rmse:0.193799+0.003755 
[3]	train-rmse:0.159236+0.000665	test-rmse:0.160641+0.003262 
[4]	train-rmse:0.149368+0.000672	test-rmse:0.151164+0.002646 
[5]	train-rmse:0.146485+0.000609	test-rmse:0.148978+0.002282 
[6]	train-rmse:0.145539+0.000707	test-rmse:0.148600+0.002136 
[7]	train-rmse:0.144936+0.000554	test-rmse:0.148552+0.001955 
[8]	train-rmse:0.144404+0.000527	test-rmse:0.148610+0.001931 
[9]	train-rmse:0.143903+0.000783	test-rmse:0.148498+0.001814 
[10]	train-rmse:0.143609+0.000784	test-rmse:0.148490+0.001813 
[11]	train-rmse:0.143053+0.000560	test-rmse:0.148822+0.001497 
[12]	train-rmse:0.142665+0.000424	test-rmse:0.149120+0.001186 
[13]	train-rmse:0.142360+0.000498	test-rmse:0.149252+0.001018 
Stopping. Best iteration:
[10]	train-rmse:0.143609+0.000784

Joining, by = "swap"



In [ ]:
saveRDS(placeboResults, '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/placebo_DiD.RDS')

In [13]:
placeboResults <- readRDS('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/placebo_DiD.RDS')
panel %>%
    mutate(alter_bgmr     = weightedAlters(., WM, ban_gmr),
           alter_sh       = weightedAlters(., WM, stay_home),
           geo_alter_bgmr = weightedAlters(., gWM, ban_gmr),
           geo_alter_sh   = weightedAlters(., gWM, stay_home)) %>%
    mutate(alter_bgmr.r     = panel_residualizer(alter_bgmr),
           alter_sh.r       = panel_residualizer(alter_sh),
           geo_alter_bgmr.r = panel_residualizer(geo_alter_bgmr),
           geo_alter_sh.r   = panel_residualizer(geo_alter_sh)) -> panel

[1]	train-rmse:0.261843+0.000594	test-rmse:0.262175+0.002129 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.146072+0.000844	test-rmse:0.146755+0.002419 
[3]	train-rmse:0.097063+0.001147	test-rmse:0.098287+0.002693 
[4]	train-rmse:0.079901+0.001315	test-rmse:0.081463+0.002403 
[5]	train-rmse:0.074771+0.001435	test-rmse:0.076656+0.002311 
[6]	train-rmse:0.073152+0.001472	test-rmse:0.075394+0.002345 
[7]	train-rmse:0.072461+0.001615	test-rmse:0.074878+0.002479 
[8]	train-rmse:0.072167+0.001595	test-rmse:0.074825+0.002350 
[9]	train-rmse:0.071909+0.001505	test-rmse:0.074731+0.002396 
[10]	train-rmse:0.071579+0.001513	test-rmse:0.074674+0.002429 
[11]	train-rmse:0.071465+0.001497	test-rmse:0.074649+0.002373 
[12]	train-rmse:0.071233+0.001453	test-rmse:0.074673+0.002331 
[13]	train-rmse:0.071048+0.001567	test-rmse:0.074710+0.002297 
[14]	train-rmse:0.070659+0.001430	test-rmse:0.074626+0.0023

In [ ]:
originalResults <- foreach(DV = c('ash_rnstu.r', 'ash_nhd.r', 'fb_btvrc.r', 'log_mcbgv.r'), .combine = rbind) %:%
        foreach(type = c('DiD', 'geoOnly', 'geoSocial'), .combine = rbind) %dopar% getCoef(panel, DV, type)

In [ ]:
originalResults %>% filter(type == 'DiD')

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)
placeboResults %>%
    filter(type == 'DiD') %>%
    ggplot(aes(x = coef)) + 
    geom_histogram(binwidth = .0005) + 
    geom_vline(aes(xintercept = coef), data = originalResults %>% filter(type == 'DiD'), color = 'red') +
    facet_grid(vars~DV)

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)
placeboResults %>%
    filter(type == 'geoOnly') %>%
    ggplot(aes(x = coef)) + 
    geom_histogram(binwidth = .0005) + 
    geom_vline(aes(xintercept = coef), data = originalResults %>% filter(type == 'geoOnly'), color = 'red') +
    facet_grid(vars~DV, scales = 'free')

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)
placeboResults %>%
    filter(type == 'geoSocial') %>%
    ggplot(aes(x = coef)) + 
    geom_histogram(binwidth = 0.001) + 
    geom_vline(aes(xintercept = coef), data = originalResults %>% filter(type == 'geoSocial'), color = 'red') +
    facet_grid(vars~DV, scales = 'free')